# Django用户注册系统

## 一，创建一个 zqxt_auth 项目

```
django-admin startproject zqxt_auth
```

打开 zqxt_auth/setting.py 可以看到 django.contrib.auth 已经在 INSTALLED_APPS 中 ：

```
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]
```

## 二，修改 urls.py 

```
from django.conf.urls import url, include
from django.contrib import admin
from django.contrib.auth import urls as auth_urls
 
 
urlpatterns = [
    url(r'^accounts/', include(auth_urls, namespace='accounts')),
    url(r'^admin/', admin.site.urls),
]
```

我们引入了 django.contrib.auth.urls 中的内容，改好后，我们试着访问一下：

http://127.0.0.1:8000/accounts/login/  

就会报错，模板不存在，然后我们要创建模板

## 三，准备相关的模板文件【此步阅读即可，不需要照着做】

Django默认配置下会自动寻找 app 下的模板，但是 django.contrib.auth 这个是官方提供的，我们修改这个 app 不太容易，我们可以建立一个公用的模板文件夹。



** 3.1 添加一个公用的放模板的文件夹 **

settings.py，修改 TEMPLATES 中的 DIRS

```
TEMPLATES = [
    {
        ...
        'DIRS': [os.path.join(BASE_DIR, 'templates')],
        ...
    },
]
```

这个文件夹需要我们手工创建出来，创建后如下：

```
.
├── db.sqlite3
├── manage.py
├── templates 
└── zqxt_auth
    ├── __init__.py
    ├── settings.py
    ├── urls.py
    └── wsgi.py
```

** 3.2 模板文件 **

熟悉Django的同学知道，Django 的后台是有登陆，重设密码的功能，那么在 django.contrib.admin 中应该是有相应的模板文件的，[查看链接](https://github.com/django/django/tree/1.9/django/contrib/admin/templates/registration)

我们把这些文件拷贝出来。

需要注意的是，有人已经按照类似的方法，做成了一个包了，地址在这里：https://github.com/mishbahr/django-users2

这个包比较完善了，我们没必要重新发明轮子，上面的示例只是让你明白，这个包其实是由官方的django.contrib.auth 改进后做出来的。

## 四，用 django-users2 这个包来实现登陆注册及找回密码等功能。

** 4.1 安装 **

```
pip install django-users2
```


** 4.2 把 users 这个 app 加入到 INSTALLED_APPS **

```
INSTALLED_APPS = (
    ...
    'django.contrib.auth',
    'django.contrib.sites',
    'users',
    ...
)
 
AUTH_USER_MODEL = 'users.User'\
```


AUTH_USER_MODEL 是替换成自定义的用户认证。[参考这里](https://docs.djangoproject.com/en/1.9/topics/auth/customizing/#auth-custom-user)



** 4.3 修改 urls.py **

```
urlpatterns = patterns('',
    ...
    url(r'^accounts/', include('users.urls')),
    ...
)
```



** 4.4 同步数据，创建相应的表 **

```
python manage.py makemigrations
python manage.py migrate
```



** 4.5 配置登陆注册的一些选项，找密码时发邮件的邮箱 **

下面的代码加在 settings.py 最后面

```
USERS_REGISTRATION_OPEN = True
 
USERS_VERIFY_EMAIL = True
 
USERS_AUTO_LOGIN_ON_ACTIVATION = True
 
USERS_EMAIL_CONFIRMATION_TIMEOUT_DAYS = 3
 
# Specifies minimum length for passwords:
USERS_PASSWORD_MIN_LENGTH = 5
 
# Specifies maximum length for passwords:
USERS_PASSWORD_MAX_LENGTH = None
 
# the complexity validator, checks the password strength
USERS_CHECK_PASSWORD_COMPLEXITY = True
 
USERS_SPAM_PROTECTION = False  # important!
 
 
#  ---------------------------------------------------------
#  Email
EMAIL_BACKEND = 'django.core.mail.backends.smtp.EmailBackend'
 
EMAIL_USE_TLS = False
EMAIL_HOST = 'smtp.tuweizhong.com'
EMAIL_PORT = 25
EMAIL_HOST_USER = 'mail@tuweizhong.com'
EMAIL_HOST_PASSWORD = 'xxxx'
DEFAULT_FROM_EMAIL = 'mail@tuweizhong.com'
#  ---------------------------------------------------------
```